## Import des librairies

In [85]:
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from pickle import dump
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, recall_score, confusion_matrix

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Chargement des données

In [5]:
data = pd.read_csv("../data/processed/processed_data.csv")
data.head()

,GP,MIN,FG%,3P Made,3P%,FTM,FT%,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,-1.400901,1.177066,-1.543407,0.658041,0.356835,0.306333,-0.037855,1.010852,0.517997,0.237640,-0.533458,0.073254,0.147338,0.0
1,-1.458281,1.116860,-2.374651,1.179494,0.262797,1.319630,0.586286,-0.018938,-0.308447,1.461613,1.175502,0.306415,0.562695,0.0
2,0.779563,-0.279912,-0.320991,0.397315,0.319219,-0.402975,-0.312099,-0.239607,-0.405676,-0.374347,-0.289321,-0.159906,-0.268019,0.0
3,-0.138527,-0.725434,-0.255795,-0.384863,0.206374,-0.402975,-0.132422,-0.828059,-0.551519,-0.510344,-0.045184,-0.626228,-0.268019,1.0
4,-0.712333,-0.737475,1.341496,-0.645589,-1.210464,0.002344,-0.274272,-0.386720,-0.259833,-0.850337,-0.777595,0.073254,-0.544923,1.0


In [6]:
X = data.drop(columns=['TARGET_5Yrs'])
y = data['TARGET_5Yrs']

## Entraînement et choix des modèles

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Entraînement des modèles 

#### 1- Logistic Regression

In [32]:
grid_lr = {"C":np.logspace(-4,0,5), 
           "penalty":["l1","l2"],
           "solver": ["lbfgs", "liblinear"]}

In [33]:
lr = LogisticRegression()
lr_cv = GridSearchCV(estimator=lr, param_grid=grid_lr, scoring="precision", cv=10)
lr_cv.fit(X_train,y_train)

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan 0.         0.63526307 0.79438574        nan 0.
 0.650018   0.78165878        nan 0.75358722 0.72546298 0.75280189
        nan 0.73860299 0.74164172 0.74371664        nan 0.73889231
 0.74096245 0.74203612]
  warnings.warn(


GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00]),
                         'penalty': ['l1', 'l2'],
                         'solver': ['lbfgs', 'liblinear']},
             scoring='precision')

In [34]:
lr_cv.best_params_

{'C': 0.0001, 'penalty': 'l2', 'solver': 'liblinear'}

In [35]:
lr_cv.best_score_

0.794385741375353

In [40]:
lr = LogisticRegression(C=0.0001, penalty='l2', solver='liblinear', random_state=0)
lr.fit(X_train,y_train)

LogisticRegression(C=0.0001, random_state=0, solver='liblinear')

#### 2- SGD Classifier

In [45]:
grid_sgd = {"loss": ["hinge", "log", "modified_huber"], 
           "penalty": ["l1","l2","elasticnet"],
           "alpha": [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1]}

In [46]:
sgd = SGDClassifier(early_stopping=True)
sgd_cv = GridSearchCV(estimator=sgd, param_grid=grid_sgd, scoring="precision", cv=10)
sgd_cv.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=SGDClassifier(early_stopping=True),
             param_grid={'alpha': [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1],
                         'loss': ['hinge', 'log', 'modified_huber'],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             scoring='precision')

In [47]:
sgd_cv.best_params_

{'alpha': 0.005, 'loss': 'hinge', 'penalty': 'elasticnet'}

In [50]:
sgd_cv.best_score_

0.7534366604436609

In [51]:
sgd = SGDClassifier(early_stopping=True,
                   random_state=0,
                   alpha=0.005,
                   loss="hinge",
                   penalty="elasticnet")
sgd.fit(X_train,y_train)

SGDClassifier(alpha=0.005, early_stopping=True, penalty='elasticnet',
              random_state=0)

#### 3- Decision Tree

In [53]:
grid_dt = {"criterion": ["gini", "entropy", "log_loss"], 
           "min_samples_split": [2, 5, 10, 20],
           "splitter": ["best", "random"]}

In [54]:
dt = DecisionTreeClassifier()
dt_cv = GridSearchCV(estimator=dt, param_grid=grid_dt, scoring="precision", cv=10)
dt_cv.fit(X_train,y_train)

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 903, in fit
    super().fit(
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 348, in fit
    criterion = CRITERIA_CLF[self.criterion](self.n_outputs_,
KeyError: 'log_loss'

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWa

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'min_samples_split': [2, 5, 10, 20],
                         'splitter': ['best', 'random']},
             scoring='precision')

In [56]:
dt_cv.best_params_

{'criterion': 'entropy', 'min_samples_split': 10, 'splitter': 'best'}

In [57]:
dt_cv.best_score_

0.7180035488969325

In [58]:
dt = DecisionTreeClassifier(random_state=0,
                           criterion="entropy",
                           min_samples_split=10,
                           splitter="best")
dt.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', min_samples_split=10,
                       random_state=0)

#### 4- Random Forest

In [62]:
grid_rf = {"criterion": ["gini", "entropy", "log_loss"], 
           "min_samples_split": [2, 5, 10, 20],
           "n_estimators": [100, 150, 200, 50, 20]}

In [63]:
rf = RandomForestClassifier()
rf_cv = GridSearchCV(estimator=rf, param_grid=grid_rf, scoring="precision", cv=10)
rf_cv.fit(X_train,y_train)

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/tahayassinebena

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/tahayassinebena

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/tahayassinebena

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/tahayassinebena

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/tahayassinebena

...

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/tahayassinebena

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/tahayassinebena

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/tahayassinebena

/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/tahayassinebenali/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/tahayassinebena

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'min_samples_split': [2, 5, 10, 20],
                         'n_estimators': [100, 150, 200, 50, 20]},
             scoring='precision')

In [64]:
rf_cv.best_score_

0.7444858099010663

In [65]:
rf_cv.best_params_

{'criterion': 'gini', 'min_samples_split': 2, 'n_estimators': 50}

In [66]:
rf = RandomForestClassifier(random_state=0,
                           criterion="gini",
                           min_samples_split=2,
                           n_estimators=50)
rf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=50, random_state=0)

#### 5- LightGBM

In [77]:
grid_lgb = {'max_depth': [3,4,5],
        'n_estimators': [100, 150, 200, 50, 20, 250],
        'learning_rate': [0.01, 0.005, 0.1, 0.05, 0.02]}

In [78]:
lgb_model = lgb.LGBMClassifier()
lgb_cv = GridSearchCV(estimator=lgb_model, param_grid=grid_lgb, scoring='precision', cv=10)
lgb_cv.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.01, 0.005, 0.1, 0.05, 0.02],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 150, 200, 50, 20, 250]},
             scoring='precision')

In [79]:
lgb_cv.best_score_

0.7372814728839389

In [80]:
lgb_cv.best_params_

{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 150}

In [95]:
lgb = lgb.LGBMClassifier(random_state=0,
                           learning_rate=0.05,
                           max_depth=4,
                           n_estimators=150)
lgb.fit(X_train,y_train)

LGBMClassifier(learning_rate=0.05, max_depth=4, n_estimators=150,
               random_state=0)

### Evaluation des modèles

On a utilisé les métriques classiques pour évaluer les modèles. On utilise maintenant la métrique proposée par MPData. 

In [86]:
def score_classifier(dataset,classifier,labels):

    """
    performs 3 random trainings/tests to build a confusion matrix and prints results with precision and recall scores
    :param dataset: the dataset to work on
    :param classifier: the classifier to use
    :param labels: the labels used for training and validation
    :return:
    """

    kf = KFold(n_splits=3,random_state=50,shuffle=True)
    confusion_mat = np.zeros((2,2))
    recall = 0
    for training_ids,test_ids in kf.split(dataset):
        training_set = dataset[training_ids]
        training_labels = labels[training_ids]
        test_set = dataset[test_ids]
        test_labels = labels[test_ids]
        classifier.fit(training_set,training_labels)
        predicted_labels = classifier.predict(test_set)
        confusion_mat+=confusion_matrix(test_labels,predicted_labels)
        recall += recall_score(test_labels, predicted_labels)
    recall/=3
    print(confusion_mat)
    print(recall)

In [90]:
print("La performance de la régression logisitique : ")
score_classifier(X.values, lr, y.values)

La performance de la régression logisitique : 
[[369. 140.]
 [299. 532.]]
0.640093544778948


In [97]:
print("La performance du classifieur SGD: ")
score_classifier(X.values, sgd, y.values)

La performance du classifieur SGD: 
[[200. 309.]
 [135. 696.]]
0.8366526085304183


In [98]:
print("La performance de l'arbre de décision : ")
score_classifier(X.values, dt, y.values)

La performance de l'arbre de décision : 
[[280. 229.]
 [282. 549.]]
0.6604282981082953


In [99]:
print("La performance de la forêt aléatoire : ")
score_classifier(X.values, rf, y.values)

La performance de la forêt aléatoire : 
[[262. 247.]
 [168. 663.]]
0.7978555851252093


In [100]:
print("La performance du LightGBM : ")
score_classifier(X.values, lgb, y.values)

La performance du LightGBM : 
[[257. 252.]
 [175. 656.]]
0.7892802914493497


Le modèle qui donne le meilleur rappel est le **`SGDClassifier`**.